In [44]:
%pip install --upgrade torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.2/808.2 kB 7.9 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


### Imports

In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image
import random

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.models as models

from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm


device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print("Device:", device)


Device: mps


### Configs

In [2]:
DATA_DIR = Path("train_images/train_images")
csv_path = Path("train_images.csv") 

IMAGE_SIZE = 224
BATCH_SIZE = 64
VAL_SPLIT = 0.1
EPOCHS = 200
LR = 5e-4
SEED = 42
NUM_CLASSES = 200

writer = SummaryWriter(log_dir="./runs/bird_experiment")  # TensorBoard logging

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("⚡ Running on Apple MPS GPU")
else:
    device = torch.device("cpu")
    print("🐌 Running on CPU")

⚡ Running on Apple MPS GPU


### Datapreperation

### Dataset class

In [3]:

class BirdDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = str(row["image_path"])  # zorg dat het string is
        label = row["label0"] if "label0" in row else row["label"]

        img = Image.open(img_path).convert("RGB")

        if self.transform:
            img = self.transform(img)

        return img, label


### Transformers

In [4]:
train_transform = T.Compose([
    T.RandomResizedCrop(IMAGE_SIZE),
    T.RandomHorizontalFlip(),
    T.RandomRotation(15),
    T.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    T.RandomAffine(degrees=15, translate=(0.1,0.1), scale=(0.9,1.1)),
    T.RandomPerspective(distortion_scale=0.1, p=0.5),  # nieuwe augmentatie
    T.RandomGrayscale(p=0.05),
    T.ToTensor(),
    T.RandomErasing(p=0.25, scale=(0.02,0.2)),  
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

test_transform = T.Compose([
    T.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    T.CenterCrop(IMAGE_SIZE),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

### Train validation split

In [5]:
# CSV inlezen
df = pd.read_csv(csv_path, header=0)
df.columns = ["image_path", "label"]

# Shuffle
df = df.sample(frac=1.0, random_state=SEED).reset_index(drop=True)

# Split in train en validation
val_size = int(len(df) * VAL_SPLIT)
df_val = df.iloc[:val_size].copy()
df_train = df.iloc[val_size:].copy()

# Corrigeer image_path zodat het naar de juiste map verwijst
df_train['image_path'] = df_train['image_path'].apply(lambda x: DATA_DIR / x.split('/')[-1])
df_val['image_path'] = df_val['image_path'].apply(lambda x: DATA_DIR / x.split('/')[-1])


# Maak labels zero-based
df_train['label'] = df_train['label'] - 1
df_val['label'] = df_val['label'] - 1

# Controleer
print("Min label:", df_train['label'].min())
print("Max label:", df_train['label'].max())


len(df_train), len(df_val)

Min label: 0
Max label: 199


(3534, 392)

### Data loaders

In [6]:
train_ds = BirdDataset(df_train, transform=train_transform)
val_ds = BirdDataset(df_val, transform=test_transform)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)

### Model setup

In [7]:
model = models.resnet18(weights=None) 
model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Training + TensorBoard Logging nieuw

In [8]:
resume = True
checkpoint_path = "last_checkpoint_best.pth"

#===========================================================
# Load attributes
#===========================================================
attributes = np.load("attributes.npy", allow_pickle=True)
attributes = torch.tensor(attributes, dtype=torch.float32).to(device)
num_attributes = attributes.shape[1]

#===========================================================
# Model uitbreiden met attribute head
#===========================================================
class BirdNet(nn.Module):
    def __init__(self, base_model, num_classes, num_attributes):
        super().__init__()
        self.backbone = base_model
        in_features = base_model.fc.in_features
        base_model.fc = nn.Identity()
        self.classifier = nn.Linear(in_features, num_classes)
        self.attribute_head = nn.Linear(in_features, num_attributes)

    def forward(self, x):
        feats = self.backbone(x)
        return self.classifier(feats), self.attribute_head(feats)

#===========================================================
# Base model + wrapper
#===========================================================
base = models.resnet18(weights=None)
model = BirdNet(base, num_classes=200, num_attributes=num_attributes).to(device)

criterion_cls = nn.CrossEntropyLoss(label_smoothing=0.1)
criterion_attr = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader)*EPOCHS)
writer = SummaryWriter()

best_acc = 0
start_epoch = 1

#===========================================================
# RESUME BLOCK
#===========================================================
if resume and os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    if "model_state" in checkpoint:
        print("🔄 Resume training...")
        model.load_state_dict(checkpoint["model_state"])
        optimizer.load_state_dict(checkpoint["optimizer_state"])
        scheduler.load_state_dict(checkpoint["scheduler_state"])
        best_acc = checkpoint["best_acc"]
        start_epoch = checkpoint["epoch"] + 1
        print(f"✔️ Resume vanaf epoch {start_epoch} | best_acc={best_acc:.4f}")
    else:
        print("⚠️ Oude checkpoint → alleen model geladen")
        model.load_state_dict(checkpoint)
        start_epoch = 1
        best_acc = 0
        print(f"✔️ Start vanaf epoch {start_epoch}")

#===========================================================
# TRAINING LOOP
#===========================================================
for epoch in range(start_epoch, EPOCHS+1):
    model.train()
    train_loss, train_loss_cls, train_loss_attr, correct, total = 0, 0, 0, 0, 0

    for imgs, labels in tqdm(train_loader, desc=f"Train {epoch}"):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        class_logits, attr_out = model(imgs)
        attr_target = attributes[labels]
        loss_cls = criterion_cls(class_logits, labels)
        loss_attr = criterion_attr(attr_out, attr_target)
        loss = loss_cls + 0.5 * loss_attr
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * imgs.size(0)
        train_loss_cls += loss_cls.item() * imgs.size(0)
        train_loss_attr += loss_attr.item() * imgs.size(0)
        correct += (class_logits.argmax(1) == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    train_loss_avg = train_loss / total
    train_loss_cls_avg = train_loss_cls / total
    train_loss_attr_avg = train_loss_attr / total

    #===========================================================
    # VALIDATION
    #===========================================================
    model.eval()
    val_loss, val_loss_cls, val_loss_attr, correct, total = 0, 0, 0, 0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            class_logits, attr_out = model(imgs)
            attr_target = attributes[labels]
            loss_cls = criterion_cls(class_logits, labels)
            loss_attr = criterion_attr(attr_out, attr_target)
            loss = loss_cls + 0.5 * loss_attr
            val_loss += loss.item() * imgs.size(0)
            val_loss_cls += loss_cls.item() * imgs.size(0)
            val_loss_attr += loss_attr.item() * imgs.size(0)
            correct += (class_logits.argmax(1) == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total
    val_loss_avg = val_loss / total
    val_loss_cls_avg = val_loss_cls / total
    val_loss_attr_avg = val_loss_attr / total

    print(f"Epoch {epoch:02d} | "
          f"Train Acc: {train_acc:.4f}, Train Loss: {train_loss_avg:.4f} "
          f"(Cls: {train_loss_cls_avg:.4f}, Attr: {train_loss_attr_avg:.4f}) | "
          f"Val Acc: {val_acc:.4f}, Val Loss: {val_loss_avg:.4f} "
          f"(Cls: {val_loss_cls_avg:.4f}, Attr: {val_loss_attr_avg:.4f})")

    #===========================================================
    # TensorBoard logging
    #===========================================================
    writer.add_scalar("Loss/train_total", train_loss_avg, epoch)
    writer.add_scalar("Loss/train_cls", train_loss_cls_avg, epoch)
    writer.add_scalar("Loss/train_attr", train_loss_attr_avg, epoch)
    writer.add_scalar("Loss/val_total", val_loss_avg, epoch)
    writer.add_scalar("Loss/val_cls", val_loss_cls_avg, epoch)
    writer.add_scalar("Loss/val_attr", val_loss_attr_avg, epoch)
    writer.add_scalar("Accuracy/train", train_acc, epoch)
    writer.add_scalar("Accuracy/val", val_acc, epoch)
    writer.add_scalar("LR", scheduler.get_last_lr()[0], epoch)
    scheduler.step()

    #===========================================================
    # SAVE + RESUME CHECKPOINT
    #===========================================================
    checkpoint_data = {
        "epoch": epoch,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "scheduler_state": scheduler.state_dict(),
        "best_acc": best_acc
    }
    torch.save(checkpoint_data, checkpoint_path)

    # Beste model apart
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model_ResNet18.pth")
        print("✔️ New best model saved")

writer.close()
print("Best validation accuracy:", best_acc)

Train 1: 100%|██████████| 56/56 [01:44<00:00,  1.87s/it]


Epoch 01 | Train Acc: 0.0091, Train Loss: 5.3646 (Cls: 5.3408, Attr: 0.0477) | Val Acc: 0.0153, Val Loss: 5.4304 (Cls: 5.3838, Attr: 0.0931)
✔️ New best model saved


Train 2: 100%|██████████| 56/56 [01:28<00:00,  1.58s/it]


Epoch 02 | Train Acc: 0.0136, Train Loss: 5.2003 (Cls: 5.1930, Attr: 0.0146) | Val Acc: 0.0128, Val Loss: 5.3281 (Cls: 5.3143, Attr: 0.0278)


Train 3: 100%|██████████| 56/56 [01:31<00:00,  1.63s/it]


...

Epoch 198 | Train Acc: 0.8079, Train Loss: 1.7104 (Cls: 1.7097, Attr: 0.0013) | Val Acc: 0.2959, Val Loss: 3.8831 (Cls: 3.8823, Attr: 0.0016)


Train 199: 100%|██████████| 56/56 [01:23<00:00,  1.50s/it]


Epoch 199 | Train Acc: 0.8084, Train Loss: 1.7199 (Cls: 1.7192, Attr: 0.0013) | Val Acc: 0.2985, Val Loss: 3.9315 (Cls: 3.9308, Attr: 0.0016)


Train 200: 100%|██████████| 56/56 [01:23<00:00,  1.50s/it]


Epoch 200 | Train Acc: 0.8164, Train Loss: 1.6901 (Cls: 1.6895, Attr: 0.0013) | Val Acc: 0.3010, Val Loss: 3.8319 (Cls: 3.8312, Attr: 0.0015)
Best validation accuracy: 0.3520408163265306


### Test and submission Merlijne nieuw


In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms as T
from PIL import Image
import pandas as pd

# =========================
# Paden en instellingen
# =========================
input_csv_path = 'test_images_path.csv'
image_folder = 'test_images/test_images'
model_path = 'best_model_ResNet18.pth'
output_csv_final = 'submission_ResNet18.csv'

device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
print("Device:", device)

# =========================
# Model-definitie (zelfde als training!)
# =========================
class BirdNet(nn.Module):
    def __init__(self, base_model, num_classes, num_attributes):
        super().__init__()
        self.backbone = base_model
        in_features = base_model.fc.in_features
        base_model.fc = nn.Identity()

        self.classifier = nn.Linear(in_features, num_classes)
        self.attribute_head = nn.Linear(in_features, num_attributes)

    def forward(self, x):
        feats = self.backbone(x)
        return self.classifier(feats), self.attribute_head(feats)

# =========================
# Test transform
# =========================
IMAGE_SIZE = 224
transform = T.Compose([
    T.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    T.CenterCrop(IMAGE_SIZE),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406],
                [0.229, 0.224, 0.225])
])

# =========================
# Laad attributen enkel voor shape
# =========================
attributes = np.load("attributes.npy", allow_pickle=True)
num_attributes = attributes.shape[1]

# =========================
# Model laden
# =========================
base = models.resnet18(weights=None)
model = BirdNet(base, num_classes=200, num_attributes=num_attributes).to(device)

state_dict = torch.load(model_path, map_location=device)
model.load_state_dict(state_dict)
model.eval()

print("Model geladen.")

# =========================
# CSV inlezen & predict
# =========================
df = pd.read_csv(input_csv_path)
results = []

for _, row in df.iterrows():
    full_path = f"{image_folder}/{row['image_path'].split('/')[-1]}"
    id_val = row['id']

    image = Image.open(full_path).convert('RGB')
    tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        class_logits, _ = model(tensor)
        pred = class_logits.argmax(1).item()
        label = pred + 1  # terug naar 1..200

    results.append((id_val, label))

# =========================
# Opslaan CSV
# =========================
df_out = pd.DataFrame(results, columns=['id', 'label'])
df_out.to_csv(output_csv_final, index=False)

print("OK → submission_ResNet18.csv aangemaakt")
print(df_out.head())


Device: mps
Model geladen.
